In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
import matplotlib.cm as cm
import seaborn as sns
from matplotlib import gridspec
import matplotlib.animation as manimation
import time
import matplotlib.patches as patches
from Bio import SeqIO
from collections import Counter
import re

In [9]:
KEGGorthology = list(open('KEGGreffiles/ko00002_mostrecent_201803.keg')) #change this after
#downloading the most recent version at http://www.genome.jp/kegg-bin/get_htext?ko00002.keg
KEGGorthology = KEGGorthology[3:-4] #check this! 

In [10]:
KEGGorthology[3:6]

['C    Carbon fixation\n',
 'D      M00165  Reductive pentose phosphate cycle (Calvin cycle) [PATH:map01200 map00710]\n',
 'E        K00855  PRK, prkB; phosphoribulokinase [EC:2.7.1.19]\n']

In [11]:
# this makes a useful df from ko00002.keg modules file 
KEGGmoddf = pd.DataFrame(columns=['A','B','C','D-module','D','E-module','E'])
j=0;a=0;b=0;c=0;d=0
KEGGmoddf.loc[0] = ["nan","nan","nan","nan","nan","nan","nan"]

for i in range(0,len(KEGGorthology)):
    if 'A<' in KEGGorthology[i]:
        KEGGmoddf.iloc[j,0] = KEGGorthology[i][4:-5]
        a=0
    elif 'B  ' in KEGGorthology[i]:
        KEGGmoddf.iloc[j,1] = KEGGorthology[i][6:-5]
        b=0
    elif 'C  ' in KEGGorthology[i]:
        KEGGmoddf.iloc[j,2] = KEGGorthology[i][5:-1]
        c=0
    elif 'D   ' in KEGGorthology[i]:
        KEGGmoddf.iloc[j,3] = KEGGorthology[i][7:13]
        KEGGmoddf.iloc[j,4] = KEGGorthology[i][15:-1]
        d=0
    elif 'E   ' in KEGGorthology[i]:
        KEGGmoddf.iloc[j,5] = KEGGorthology[i][9:15]
        KEGGmoddf.iloc[j,6] = KEGGorthology[i][17:-1]

        if a==1 and b==1 and c==1 and d==1: #no new a, b, c, d
            KEGGmoddf.iloc[j,0] = KEGGmoddf.iloc[j-1,0]
            KEGGmoddf.iloc[j,1] = KEGGmoddf.iloc[j-1,1]
            KEGGmoddf.iloc[j,2] = KEGGmoddf.iloc[j-1,2]
            KEGGmoddf.iloc[j,3] = KEGGmoddf.iloc[j-1,3]
            KEGGmoddf.iloc[j,4] = KEGGmoddf.iloc[j-1,4]           
        if a==1 and b==1 and c==1 and d==0: # no new a, b, c
            KEGGmoddf.iloc[j,0] = KEGGmoddf.iloc[j-1,0]
            KEGGmoddf.iloc[j,1] = KEGGmoddf.iloc[j-1,1]
            KEGGmoddf.iloc[j,2] = KEGGmoddf.iloc[j-1,2]            
        if a==1 and b==1 and c==0 and d==0: # no new a, b
            KEGGmoddf.iloc[j,0] = KEGGmoddf.iloc[j-1,0]
            KEGGmoddf.iloc[j,1] = KEGGmoddf.iloc[j-1,1]
        if a==1 and b==0 and c==0 and d==0: # no new a
            KEGGmoddf.iloc[j,0] = KEGGmoddf.iloc[j-1,0]            
        a=1;b=1;c=1;d=1
        j+=1
        if i!=len(KEGGorthology):
            KEGGmoddf.loc[j] = ["nan","nan","nan","nan","nan","nan","nan"]
KEGGmoddf.to_pickle('KEGG_modules_df')

In [12]:
KEGGmoddf

,A,B,C,D-module,D,E-module,E
0,Pathway module,Energy metabolism,Carbon fixation,M00165,Reductive pentose phosphate cycle (Calvin cycl...,K00855,"PRK, prkB; phosphoribulokinase [EC:2.7.1.19]"
1,Pathway module,Energy metabolism,Carbon fixation,M00165,Reductive pentose phosphate cycle (Calvin cycl...,K01601,rbcL; ribulose-bisphosphate carboxylase large ...
2,Pathway module,Energy metabolism,Carbon fixation,M00165,Reductive pentose phosphate cycle (Calvin cycl...,K01602,rbcS; ribulose-bisphosphate carboxylase small ...
3,Pathway module,Energy metabolism,Carbon fixation,M00165,Reductive pentose phosphate cycle (Calvin cycl...,K00927,"PGK, pgk; phosphoglycerate kinase [EC:2.7.2.3]"
4,Pathway module,Energy metabolism,Carbon fixation,M00165,Reductive pentose phosphate cycle (Calvin cycl...,K05298,GAPA; glyceraldehyde-3-phosphate dehydrogenase...
5,Pathway module,Energy metabolism,Carbon fixation,M00165,Reductive pentose phosphate cycle (Calvin cycl...,K00150,gap2; glyceraldehyde-3-phosphate dehydrogenase...
6,Pathway module,Energy metabolism,Carbon fixation,M00165,Reductive pentose phosphate cycle (Calvin cycl...,K00134,"GAPDH, gapA; glyceraldehyde 3-phosphate dehydr..."
7,Pathway module,Energy metabolism,Carbon fixation,M00165,Reductive pentose phosphate cycle (Calvin cycl...,K01623,"ALDO; fructose-bisphosphate aldolase, class I ..."
8,Pathway module,Energy metabolism,Carbon fixation,M00165,Reductive pentose phosphate cycle (Calvin cycl...,K01624,"FBA, fbaA; fructose-bisphosphate aldolase, cla..."
9,Pathway module,Energy metabolism,Carbon fixation,M00165,Reductive pentose phosphate cycle (Calvin cycl...,K03841,"FBP, fbp; fructose-1,6-bisphosphatase I [EC:3...."


In [13]:
#get list of of modules from the KEGG database 
from urllib.request import Request, urlopen, URLError
start_time = time.time()  
request = Request('http://rest.kegg.jp/list/module/')

try:
    response = urlopen(request)
    kittens = response.read()
    print(kittens[559:1000])
except (URLError):
    print('No kittez. Got an error code')
    
kittens2 = kittens.decode('utf-8')
mods = kittens2.split('\n')
mods = [re.split('\t',i) for i in mods]
mod_df_temp = pd.DataFrame.from_records(mods,columns=['typeModule','Module_long'])#,'kingdom','class','order'])
modtemp = mod_df_temp['typeModule'].str.split(':', 1, expand=True)
mod_df = modtemp.join(mod_df_temp[['Module_long']])
mod_df.columns = ['type','Module','Module_long']
mod_df = mod_df.drop('type',1);mod_df = mod_df.iloc[0:-1,:]
end_time = time.time()  
print('Retrieving module list from KEGG database took {:.2f} s'.format(end_time - start_time)) 
mod_df['Module_combined'] = mod_df['Module']+', '+mod_df['Module_long']
mod_df.to_pickle('KEGG_module_list')

b'cycle (TCA cycle, Krebs cycle)\nmd:M00010\tCitrate cycle, first carbon oxidation, oxaloacetate => 2-oxoglutarate\nmd:M00011\tCitrate cycle, second carbon oxidation, 2-oxoglutarate => oxaloacetate\nmd:M00012\tGlyoxylate cycle\nmd:M00013\tMalonate semialdehyde pathway, propanoyl-CoA => acetyl-CoA\nmd:M00014\tGlucuronate pathway (uronate pathway)\nmd:M00015\tProline biosynthesis, glutamate => proline\nmd:M00016\tLysine biosynthesis, succinyl-DAP pathway,'
Retrieving module list from KEGG database took 1.88 s


In [45]:
#get list of organisms
from urllib.request import Request, urlopen, URLError

request = Request('http://rest.kegg.jp/list/organism/')

try:
    response = urlopen(request)
    kittens = response.read()
    print(kittens[559:1000])
except (URLError):
    print('No kittez. Got an error code')
    
kittens2 = kittens.decode('utf-8')
database = kittens2.split('\n')
database = np.asarray(database)
database2 = [i.split('\t') for i in database]
database2 = pd.DataFrame.from_records(database2,columns=['ID','code','species','domain'])#,'kingdom','class','order'])
database3 = database2['domain'].str.split(';', 3, expand=True)
database3.columns = ['domain','kingdom','phylum','class']
database4 = database2[['ID','code','species']].join(database3);database4 = database4.iloc[0:-1,:]
database4.to_pickle('KEGG_organism_list')

b's;Animals;Vertebrates;Mammals\nT02918\tmcf\tMacaca fascicularis (crab-eating macaque)\tEukaryotes;Animals;Vertebrates;Mammals\nT04361\tcsab\tChlorocebus sabaeus (green monkey)\tEukaryotes;Animals;Vertebrates;Mammals\nT03989\trro\tRhinopithecus roxellana (golden snub-nosed monkey)\tEukaryotes;Animals;Vertebrates;Mammals\nT04641\trbb\tRhinopithecus bieti (black snub-nosed monkey)\tEukaryotes;Animals;Vertebrates;Mammals\nT03264\tcjc\tCallithrix jacchus (white'


In [ ]:
kinglist = database4['kingdom'].value_counts().index
kinglist
database4['kingdom'].value_counts()
kinglist = database4['kingdom'].value_counts().index
known_gen_color = known_genomes.copy()
kingcol = [-1,-.66,-.33,.33,.66,1]
a=0
for s in kinglist[[1,0,5,3,4,2]]:
    known_gen_color.ix[:,database4[database4['kingdom']==s].set_index('code').index] = known_gen_color.ix[:,database4[database4['kingdom']==s].set_index('code').index].multiply(kingcol[a])
    a+=1
known_gen_color

In [48]:
# create modules vs. known genomes df
# WARNING: this takes about 1s for every genome in the database, so around 5k s at the time of current writing (Oct 2017)
# actually 6.8ks last time (2017-10-10)
known_genomes = pd.DataFrame(index=list(mod_df['Module']),columns=list(database4['code']))
start_time = time.time()
# fill df with module 'hits' for each organism/genome
for s in known_genomes.columns:
    request = Request('http://rest.kegg.jp/list/module/'+s)
    try:
        response = urlopen(request)
        kittens = response.read()
        print(s)
    except (URLError):
        print('No kittez. Got an error code')
    kittens2 = kittens.decode('utf-8')
    org = kittens2.split('\n')
    org2 = [re.split('_|\t',i) for i in org]
    orgtemp = pd.DataFrame.from_records(org2,columns=['typecode','Module','Module_long'])
    orgtemp = orgtemp.iloc[0:-1,:]
    known_genomes.loc[orgtemp.set_index('Module').index, s] = 1
end_time = time.time()  
print('Retrieving module list from KEGG database took {:.2f} s'.format(end_time - start_time)) 
known_genomes.to_pickle('known_genomes_KEGG_module_database')

hsa
ptr
pps
ggo
pon
nle
mcc
mcf
csab
rro
rbb
cjc
sbq
mmu
rno
cge
ngi
hgl
ccan
ocu
tup
cfa
aml
umr
fca
ptg
aju
bta
bom
biu
phd
chx
oas
ssc
cfr
cdk
bacu
lve
ecb
epz
eai
myb
myd
hai
rss
pale
lav
mdo
shr
oaa
gga
mgp
cjo
apla
acyg
tgu
gfr
fab
phi
pmaj
ccw
fpg
fch
clv
aam
asn
amj
pss
cmy
cpic
acs
pvt
pbi
gja
xla
xtr
npr
dre
srx
sanh
sgh
ipu
No kittez. Got an error code
tru
tng
lco
ncc
mze
ola
xma
csem
lcf
hcq
bpec
sasa
els
sfm
lcm
cmk
bfo
cin
spu
No kittez. Got an error code
sko
dme
dpo
dan
der
dpe
dse
dsi
dwi
dya
dgr
dmo
dvi
mde
aga
aag
cqu
ame
bim
bter
soc
aec
acep
pbar
hst
cfo
lhu
pgc
nvi
tca
dpa
nvl
bmor
dpl
pxy
api
dnx
phu
fcd
dpx
isc
tut
cel
cbr
nai
bmy
loa
tsp
hro
lgi
crg
No kittez. Got an error code
obi
lak
smm
shx
ovi
nve
No kittez. Got an error code
adf
hmg
tad
aqu
ath
aly
crb
csat
eus
brp
bna
boe
thj
cit
cic
tcc
gra
ghi
egr
gmx
pvu
vra
var
ccaj
mtr
cam
adu
aip
lja
lang
fve
pper
pmum
mdm
pxb
zju
csv
cmo
No kittez. Got an error code
rcu
jcu
pop
vvi
sly
spen
sot
cann
nta
No kittez. G

cjm
cjs
cjp
cjej
cjeu
cjen
cjei
cjer
cjv
cjy
cjq
cjl
cjw
cjr
cjd
cjz
cjx
cff
cft
cfv
cfx
cfz
camp
cfp
ccv
cha
cco
ccoc
cla
clr
clm
clq
cln
cll
ccol
ccc
ccq
ccf
ccy
ccoi
ccof
ccoo
caj
cis
cvo
cpel
camr
csm
csf
cgra
cure
chyo
chv
cspf
cpin
ccun
No kittez. Got an error code
abu
abt
abl
ant
arc
alp
sdl
sba
smul
shal
suls
nsa
nis
sun
slh
nam
gsu
gsk
gme
gur
glo
gbm
geo
gem
geb
gpi
gao
gsb
pca
ppd
pace
pef
des
deu
dvu
dvl
dvm
dvg
dde
dds
ddn
dma
dsa
daf
dhy
dgg
dfi
dpg
das
dpi
dej
lip
lir
dba
doa
drt
dps
dak
dpr
dsf
dol
dml
dal
dat
dto
ade
acp
afw
ank
mxa
mfu
msd
mym
ccx
sur
age
No kittez. Got an error code
No kittez. Got an error code
vin
scl
scu
ccro
samy
llu
mrm
hoh
sat
dao
dti
sfu
dbr
hmr
dav
dpb
bba
bbat
bbw
bbac
bex
bmx
rpr
rpo
rpw
rpz
rpg
rps
rpv
rpq
rpl
rpn
rty
rtt
rtb
rcm
rcc
rbe
rbo
rco
rfe
rak
rri
rrj
rra
rrc
rrh
rrb
rrn
rrp
rrm
rrr
rms
rmi
rpk
raf
rhe
rja
rsv
rsw
rph
rau
rmo
rpp
rre
ram
rab
rmc
ots
ott
wol
wri
wen
wed
wpi
wbm
woo
wcl
ama
amf
amw
amp
acn
aov
aph
apy
apd
apha
eru
e

dtm
cbq
sco
sma
sgr
sgb
scb
ssx
svl
sct
scy
sfa
sbh
shy
sho
sve
sdv
salb
sals
strp
sfi
sci
src
salu
sall
slv
sgu
svt
stre
scw
sld
sxi
strm
strc
samb
spri
scz
scx
srw
strf
sle
srn
spav
slc
strt
sclf
sgs
stsi
sls
snr
splu
strd
snw
sauo
ksk
kab
kau
twh
tws
lxx
lxy
cmi
cms
cmc
cmh
mts
mim
mio
mix
mip
mcw
mpal
mih
micr
maur
rla
rpla
rtx
rtc
rtn
cum
cub
mvd
frp
agy
cart
cry
cphy
art
arr
arm
ari
arl
are
aaq
arh
ary
arw
arz
aru
arq
aau
ach
apn
psul
aai
gar
rsa
krh
kpl
kfv
mlu
rmu
rdn
satk
nae
bcv
bfa
brx
brv
dva
jde
kse
dni
lmoi
xce
iva
ido
cet
cceu
ske
cfl
cfi
cga
cez
ica
ars
serj
jte
bly
blin
dco
pac
pak
pav
pax
paz
paw
pad
pcn
pacc
pach
pacn
cacn
pra
pfr
pfre
prl
ppc
pbo
aaci
mph
tfl
tfa
tes
tez
nca
ndk
kfl
psim
aer
tfu
nda
nal
ngv
tcu
sro
noa
fra
fre
fri
fal
fsy
ace
nml
gob
bsd
mmar
kra
sen
svi
amd
amn
amm
amz
aoi
aja
amq
alu
pdx
psea
psee
pseh
pseq
pecq
phh
ami
No kittez. Got an error code
sesp
kal
kphy
led
ahm
acti
acad
ahg
all
pmad
stp
saq
mau
mil
vma
ams
ase
actn
afs
plk
cai
sna
ahe
mc

In [54]:
#make module index dataframe, long and short version of a module 
keggindex = pd.DataFrame(index=KEGGmoddf.index,columns=['D','D-module'])
keggindex['D'] = KEGGmoddf['D']
keggindex['D-module'] = KEGGmoddf['D-module']
keggindex['long'] = KEGGmoddf['D-module']+', '+KEGGmoddf['D']
keggindex2 = [i.split(' [P',1)[0] for i in keggindex['long']]
keggindex['longcut'] = keggindex2
indexswitcher = pd.DataFrame(index=list(keggindex.longcut.unique()),columns=['shorthand'])
indexswitcher['shorthand'] = indexswitcher.index.str[0:6]
indexswitcher = indexswitcher[0:-1] # check this 

In [87]:
indexswitcher.to_pickle('ObsCom_indexswitcher')